# Nephrotoxic agents

- IV Contrast 
    - look in `treatment` table
- Aminoglycosides, NSAIDs, calcineurin inhibitors, vasopressors
    - look for regex in drugName to identify HiCL code

## 0. Setup

In [65]:
# Must install pandas-gbq. Link: https://pandas-gbq.readthedocs.io/en/latest/install.html#pip
import math
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

project_id='lcp-internal'

# Display all rows
pd.set_option("display.max_rows", None)

# Helper function to read data from BigQuery into pandas dataframes.
def run_query(query):
    return pd.io.gbq.read_gbq(query,
                              project_id=project_id, verbose=False,
                              dialect='standard')

### 1. Find data which has the drug name

In [57]:
def hicl_query(*args):
    if args:
        where_clause = f" WHERE drughiclseqno IN {args}"
    query = f"""
    select
        drughiclseqno, drugname, count(*) as n
    from `physionet-data.eicu_crd.medication` med
    {where_clause}
    group by drugname, drughiclseqno
    order by drughiclseqno, n desc
    """
    return run_query(query)

def drug_query(compound, *args):
    where_clause = f"WHERE lower(drugname) like '%{compound}%'"
    if args:
        for a in args:
            where_clause += f" OR lower(drugname) like '%{a}%'"
    query = f"""
    select
        drughiclseqno, drugname, count(*) as n
    from `physionet-data.eicu_crd.medication` med
    {where_clause}
    group by drugname, drughiclseqno
    order by drughiclseqno, n desc
    """
    return run_query(query)

##### Aminoglycosides

In [21]:
# cohort with exclusions applied
med = drug_query('amikacin', 'gentamicin', 'streptomycin', 'neomycin', 'tobramycin')
print(med.shape)
display(med)

Downloading: 0rows [00:00, ?rows/s]

(0, 3)


,drughiclseqno,drugname,n


##### NSAIDS

In [25]:
med = drug_query('aspirin', 'ibuprofen', 'diclofenac', 'ketorolac', 'celecoxib', 
                 'indomethacin', 'naproxen', 'meloxicam', 'piroxicam')
print(med.shape)
display(med)
print(set(med['drughiclseqno']))

Downloading: 100%|██████████| 56/56 [00:00<00:00, 224.79rows/s]

(56, 3)


,drughiclseqno,drugname,n
0,1820.0,ASPIRIN 81 MG PO CHEW,9509
1,1820.0,ASPIRIN,9167
2,1820.0,aspirin,8447
3,1820.0,ASPIRIN 325 MG PO TABS,5075
4,1820.0,ASPIRIN EC 81 MG PO TBEC,4717
5,1820.0,ASPIRIN (BABY ASA) 81 MG TAB,4151
6,1820.0,ASPIRIN EC,2950
7,1820.0,"ASPIRIN, CHILDREN'S",2881
8,1820.0,ASPIRIN 81 MG EC TAB,2508
9,1820.0,ASPIRIN 325MG TAB,1920


{nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 3723.0, nan, nan, nan, nan, 1820.0, nan, nan, nan, nan, nan, nan, 18084.0, nan, nan, nan, nan, 5175.0, 1794.0, 21996.0}


In [82]:
med = hicl_query(3723, 1820, 18084, 5175, 1794, 21996)
print(med.shape)
display(med)

Downloading: 100%|██████████| 81/81 [00:00<00:00, 123.27rows/s]

(81, 3)


,drughiclseqno,drugname,n
0,1794,ASPIRIN,1
1,1794,None,1
2,1820,None,18608
3,1820,ASPIRIN 81 MG PO CHEW,9509
4,1820,ASPIRIN,9167
5,1820,aspirin,8447
6,1820,ASPIRIN 325 MG PO TABS,5075
7,1820,ASPIRIN EC 81 MG PO TBEC,4717
8,1820,ASPIRIN (BABY ASA) 81 MG TAB,4151
9,1820,ECOTRIN EC,3967


18084, 21996 corresponds to many drugs  

Final NSAID query: 
    
```WHERE drughiclseqno IN (1820, 3723, 5175) 
    OR LOWER(drugname) LIKE '%aspirin%' 
    OR LOWER(drugname) LIKE '%ibuprofen%' 
    OR LOWER(drugname) LIKE '%ketorolac%'
    OR LOWER(drugname) LIKE '%ecotrin%'
    OR LOWER(drugname) LIKE '%motrin%'
    OR LOWER(drugname) LIKE '%toradol%'```

##### Calcineurin inhibitors

In [52]:
med = drug_query('cyclosporine', 'tacrolimus', 'pimecrolimus')
print(med.shape)
display(med)
print(set(med['drughiclseqno']))

Downloading: 100%|██████████| 6/6 [00:00<00:00, 29.75rows/s]

(6, 3)


,drughiclseqno,drugname,n
0,20974.0,TACROLIMUS,1401
1,20974.0,TACROLIMUS 1 MG PO CAPS,758
2,8974.0,TACROLIMUS 1 MG PO CAPS,641
3,NaN,TACROLIMUS 1 MG PO CAPS,248
4,NaN,TACROLIMUS,11
5,8974.0,TACROLIMUS,1


{nan, nan, 20974.0, 8974.0}


In [53]:
med = hicl_query(20974, 8974)
print(med.shape)
display(med)

Downloading: 100%|██████████| 6/6 [00:00<00:00, 16.27rows/s]

(6, 3)


,drughiclseqno,drugname,n
0,20974,None,2458
1,20974,TACROLIMUS,1401
2,20974,TACROLIMUS 1 MG PO CAPS,758
3,8974,TACROLIMUS 1 MG PO CAPS,641
4,8974,None,178
5,8974,TACROLIMUS,1


Final calcineurin inhibitor query

```WHERE LOWER(drugname) LIKE '%tacrolimus%'
    OR drughiclseqno = 20974 
    OR drughiclseqno = 8974
```

##### Vasopressors

In [58]:
med = drug_query('phenylephrine', 'epinephrine', 'norepinephrine', 'dopamine')
print(med.shape)
display(med)
print(set(med['drughiclseqno']))

Downloading: 100%|██████████| 40/40 [00:00<00:00, 156.06rows/s]

(40, 3)


,drughiclseqno,drugname,n
0,NaN,PHENYLEPHRINE 10 MG/250 ML NS,1064
1,NaN,NOREPINEPHRINE 4 MG/250 ML NS,894
2,NaN,PHENYLEPHRINE,580
3,NaN,NOREPINEPHRINE 4 MG/250 ML NS INFUSION (STD CO...,248
4,NaN,NOREPINEPHRINE 8 MG in 250mL NS,189
5,NaN,NOREPINEPHRINE,155
6,NaN,EPINEPHrine,106
7,NaN,250 ML PLAS CONT : NOREPINEPHRINE IN D5W 8 MG/...,102
8,NaN,PHENYLEPHRINE 40 MG in 250 mL NS,59
9,NaN,EPINEPHrine 8 MG in 250 mL NS,47


{nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 2059.0, 2060.0, 18084.0, 2087.0, 34361.0, 35517.0, 2050.0, 2051.0, 36437.0, 35587.0, 36346.0}


In [59]:
med = hicl_query(2050, 2051, 35587, 2059, 2060, 18084, 2087, 34361, 35517, 36437, 36346)
print(med.shape)
display(med)

Downloading: 100%|██████████| 73/73 [00:00<00:00, 276.01rows/s]

(73, 3)


,drughiclseqno,drugname,n
0,2050,None,6464
1,2050,EPINEPHrine,2174
2,2051,None,14220
3,2051,NOREPINEPHRINE BITARTRATE 1 MG/ML IJ SOLN,2135
4,2051,NOREPINEPHRINE,1934
5,2051,NOREPINEPHRINE BITARTRATE 1 MG/ML IV SOLN,1919
6,2051,<<norepinephrine 4 mg/4 mL Inj,1736
7,2051,NOREPINEPHRINE 4 MG/250 ML NS,1475
8,2051,NOREPINEPHRINE BITARTRATE 1 MG/ML IV : 4 ML,1335
9,2051,NOREPINEPHRINE BITARTRATE 1 MG/ML AMP,1225


Not 18084

Final query for vasopressors

```WHERE drughiclseqno IN (2050, 2051, 2059, 2060, 2087, 34361, 35517, 35587, 36346, 36437) 
    OR LOWER(drugname) LIKE '%norepinephrine%' 
    OR LOWER(drugname) LIKE '%epinephrine%'
    OR LOWER(drugname) LIKE '%dopamine%'
    OR LOWER(drugname) LIKE '%phenylephrine%'
```

##### IV Contrast

In [73]:
query = """
SELECT treatmentstring, count(*) AS n
FROM `physionet-data.eicu_crd.treatment` trt
WHERE LOWER(treatmentString) LIKE '%contrast%'
GROUP BY treatmentstring
ORDER BY n DESC
"""
med = run_query(query)
print(med.shape)
display(med)

Downloading: 100%|██████████| 15/15 [00:00<00:00, 81.87rows/s]

(15, 2)


,treatmentstring,n
0,neurologic|procedures / diagnostics|head CT sc...,7008
1,pulmonary|radiologic procedures / bronchoscopy...,1614
2,pulmonary|radiologic procedures / bronchoscopy...,1477
3,"gastrointestinal|radiology, diagnostic and pro...",1385
4,"gastrointestinal|radiology, diagnostic and pro...",1169
5,neurologic|procedures / diagnostics|head CT sc...,861
6,burns/trauma|trauma|CT scan|without contrast,474
7,burns/trauma|trauma|CT scan|with contrast,270
8,infectious diseases|procedures|CT scan|with co...,106
9,infectious diseases|procedures|CT scan|without...,104


In [77]:
for each in med['treatmentstring']: print(each)

neurologic|procedures / diagnostics|head CT scan|without contrast
pulmonary|radiologic procedures / bronchoscopy|CT scan|with contrast
pulmonary|radiologic procedures / bronchoscopy|CT scan|without contrast
gastrointestinal|radiology, diagnostic and procedures|CT scan|without IV contrast
gastrointestinal|radiology, diagnostic and procedures|CT scan|with IV contrast
neurologic|procedures / diagnostics|head CT scan|with contrast
burns/trauma|trauma|CT scan|without contrast
burns/trauma|trauma|CT scan|with contrast
infectious diseases|procedures|CT scan|with contrast
infectious diseases|procedures|CT scan|without contrast
renal|medications|n-acetylcysteine|for radiocontrast dye prophylaxis
hematology|oncological issues|CT scan|with contrast
hematology|oncological issues|CT scan|without contrast
endocrine|diagnostic studies|CT scan|with contrast
endocrine|diagnostic studies|CT scan|without contrast


```WHERE treatmentstring IN (
'pulmonary|radiologic procedures / bronchoscopy|CT scan|with contrast'
, 'gastrointestinal|radiology, diagnostic and procedures|CT scan|with IV contrast',
, 'neurologic|procedures / diagnostics|head CT scan|with contrast'
, 'burns/trauma|trauma|CT scan|with contrast'
, 'infectious diseases|procedures|CT scan|with contrast'
, 'hematology|oncological issues|CT scan|with contrast'
, 'endocrine|diagnostic studies|CT scan|with contrast')```